In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths

from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

In [6]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('/content/drive/MyDrive/DeepLearning/6. Technique/dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [7]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [8]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [9]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# CNN 

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

'''
s = Sequential()
s.add(Dense(256))
s.add(Dropout())
s.add(Dense(17))
'''

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

# Output layer với softmax activation
fcHead = Dense(17, activation='softmax')(fcHead)

# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = Model(inputs=baseModel.input, outputs=fcHead)

58900480/58889256 [==============================] - 1s 0us/step


In [10]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [11]:
# augmentation cho training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [12]:
# freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
    
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 25
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
34/34 [==============================] - 61s 369ms/step - loss: 8.6167 - accuracy: 0.1145 - val_loss: 2.0872 - val_accuracy: 0.3047
Epoch 2/25
34/34 [==============================] - 12s 358ms/step - loss: 2.3152 - accuracy: 0.2862 - val_loss: 1.7794 - val_accuracy: 0.4531
Epoch 3/25
34/34 [==============================] - 12s 358ms/step - loss: 1.9257 - accuracy: 0.3757 - val_loss: 1.3569 - val_accuracy: 0.5625
Epoch 4/25
34/34 [==============================] - 12s 358ms/step - loss: 1.6747 - accuracy: 0.4602 - val_loss: 1.1480 - val_accuracy: 0.6484
Epoch 5/25
34/34 [==============================] - 12s 355ms/step - loss: 1.4781 - accuracy: 0.5248 - val_loss: 1.0677 - val_accuracy: 0.6875
Epoch 6/25
34/34 [==============================] - 12s 357ms/step - loss: 1.4095 - accuracy: 0.5376 - val_loss: 0.9036 - val_accuracy: 0.7227
Epoch 7/25
34/34 [==============================] - 12s 355ms/step - loss: 1.2817 - accuracy: 0.5822 - val_loss: 0.9484 - val_accuracy: 0.7070

In [13]:
# unfreeze some last CNN layer:
for layer in baseModel.layers[15:]:
    layer.trainable = True

numOfEpoch = 35
opt = SGD(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/35
34/34 [==============================] - 16s 379ms/step - loss: 0.6351 - accuracy: 0.7732 - val_loss: 0.3415 - val_accuracy: 0.9062
Epoch 2/35
34/34 [==============================] - 13s 374ms/step - loss: 0.4918 - accuracy: 0.8231 - val_loss: 0.3387 - val_accuracy: 0.9062
Epoch 3/35
34/34 [==============================] - 13s 369ms/step - loss: 0.4653 - accuracy: 0.8418 - val_loss: 0.3380 - val_accuracy: 0.8945
Epoch 4/35
34/34 [==============================] - 13s 371ms/step - loss: 0.4414 - accuracy: 0.8648 - val_loss: 0.3398 - val_accuracy: 0.9023
Epoch 5/35
34/34 [==============================] - 13s 373ms/step - loss: 0.3888 - accuracy: 0.8771 - val_loss: 0.3121 - val_accuracy: 0.9219
Epoch 6/35
34/34 [==============================] - 13s 374ms/step - loss: 0.4297 - accuracy: 0.8488 - val_loss: 0.2673 - val_accuracy: 0.9219
Epoch 7/35
34/34 [==============================] - 13s 368ms/step - loss: 0.3816 - accuracy: 0.8603 - val_loss: 0.3187 - val_accuracy: 0.9062